# Redes neuronales hibridas  para clasificación multiple


In [ ]:
# Install the relevant packages.

#pip install matplotlib torch torchvision torchaudio scikit-learn -q --extra-index-url https://download.pytorch.org/whl/cu118
#!pip install cudaq

In [ ]:
# Check installed versions
import sys
import torch
import numpy as np
import matplotlib



print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

#esto produce error 
#import cudaq
#print(f"CUDAQ version: {cudaq.__version__}")
#print(f"Running on target: {cudaq.get_target().name}")

Python version: 3.12.12 (main, Oct 10 2025, 00:00:00) [GCC 15.2.1 20250808 (Red Hat 15.2.1-1)]
PyTorch version: 2.9.0+cpu
NumPy version: 2.2.6
Matplotlib version: 3.10.7
CUDA available: False


In [ ]:
import sys
import cudaq

print(f"Running on target {cudaq.get_target().name}")
qubit_count = int(sys.argv[1]) if 1 < len(sys.argv) else 2


@cudaq.kernel
def kernel():
    qubits = cudaq.qvector(qubit_count)
    h(qubits[0])
    for i in range(1, qubit_count):
        x.ctrl(qubits[0], qubits[i])
    mz(qubits)


result = cudaq.sample(kernel)
print(result)  # Example: { 11:500 00:500 }

In [ ]:
# importar las librerías necesarias
import cudaq
from cudaq import spin

import matplotlib.pyplot as plt

import numpy as np

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torchvision

from sklearn.model_selection import train_test_split

torch.manual_seed(22)
cudaq.set_random_seed(44)

In [ ]:
# Configurar el dispositivo
# Set CUDAQ and PyTorch to run on either CPU or GPU.

device = torch.device('cpu')
cudaq.set_target("qpp-cpu")

#cudaq.set_target("nvidia")
#device = torch.device("cuda:0")

## Descripción del Conjunto de Datos MNIST

- *¿Qué es?:* MNIST (Modified National Institute of Standards and Technology database) es una gran base de datos de dígitos escritos a mano, del 0 al 9.
- *Contenido:* Contiene 70,000 imágenes en escala de grises.
- *Conjunto de entrenamiento:* 60,000 imágenes.
- *Conjunto de prueba:* 10,000 imágenes.
- *Formato de imagen:* Cada imagen tiene un tamaño de 28x28 píxeles.
- *Uso común:* Es considerado el "Hola, Mundo" de la visión por computadora y el aprendizaje profundo. Se utiliza para entrenar y probar algoritmos de clasificación de imágenes.


In [ ]:
def dibujar_digitos_aleatorios(dataset, n_digitos=6):
    """
    Selecciona, dibuja y etiqueta 'n_digitos' imágenes aleatorias de un dataset dado.

    Args:
        dataset: El conjunto de datos de PyTorch (e.g., train_dataset de MNIST).
        n_digitos (int): El número de dígitos aleatorios a mostrar.
    """
    # 1. Generar índices aleatorios sin repetición
    num_muestras = len(dataset)
    indices_aleatorios = np.random.choice(num_muestras, size=n_digitos, replace=False)

    # 2. Preparar la visualización
    # Creamos una figura con 'n_digitos' subtramas en una fila
    fig, axes = plt.subplots(1, n_digitos, figsize=(12, 3))
    fig.suptitle(f'{n_digitos} Dígitos Aleatorios del Dataset MNIST', fontsize=16)

    vector_etiquetas = []

    # 3. Iterar sobre los índices aleatorios para obtener y dibujar las imágenes
    for i, idx in enumerate(indices_aleatorios):
        # Obtenemos la imagen y su etiqueta del dataset
        imagen, etiqueta = dataset[idx]
        
        # Guardamos la etiqueta en nuestro vector
        vector_etiquetas.append(etiqueta)

        # Configuramos la subtrama actual
        ax = axes[i]
        # La imagen es un tensor de [1, 28, 28], usamos squeeze() para convertirla a [28, 28]
        # para que matplotlib pueda mostrarla.
        ax.imshow(imagen.squeeze(), cmap='gray')
        ax.set_title(f"Etiqueta: {etiqueta}")
        ax.axis('off') # Ocultamos los ejes para una mejor visualización

    plt.show()

    # 4. Imprimir el vector de etiquetas
    print(f"\nVector de etiquetas correspondiente a las imágenes mostradas:")
    print(vector_etiquetas)

In [ ]:
def dibujar_digitos_aleatorios(dataset, n_digitos=6):
    """
    Selecciona, dibuja y etiqueta 'n_digitos' imágenes aleatorias de un dataset dado.

    Args:
        dataset: El conjunto de datos de PyTorch (e.g., train_dataset de MNIST).
        n_digitos (int): El número de dígitos aleatorios a mostrar.
    """
    # 1. Generar índices aleatorios sin repetición
    num_muestras = len(dataset)
    indices_aleatorios = np.random.choice(num_muestras, size=n_digitos, replace=False)

    # 2. Preparar la visualización
    # Creamos una figura con 'n_digitos' subtramas en una fila
    fig, axes = plt.subplots(1, n_digitos, figsize=(12, 3))
    fig.suptitle(f'{n_digitos} Dígitos Aleatorios del Dataset MNIST', fontsize=16)

    vector_etiquetas = []

    # 3. Iterar sobre los índices aleatorios para obtener y dibujar las imágenes
    for i, idx in enumerate(indices_aleatorios):
        # Obtenemos la imagen y su etiqueta del dataset
        imagen, etiqueta = dataset[idx]
        
        # Guardamos la etiqueta en nuestro vector
        vector_etiquetas.append(etiqueta)

        # Configuramos la subtrama actual
        ax = axes[i]
        # La imagen es un tensor de [1, 28, 28], usamos squeeze() para convertirla a [28, 28]
        # para que matplotlib pueda mostrarla.
        ax.imshow(imagen.squeeze(), cmap='gray')
        ax.set_title(f"Etiqueta: {etiqueta}")
        ax.axis('off') # Ocultamos los ejes para una mejor visualización

    plt.show()

    # 4. Imprimir el vector de etiquetas
    print(f"\nVector de etiquetas correspondiente a las imágenes mostradas:")
    print(vector_etiquetas)